In [1]:
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.transforms import NormalizeFeatures

In [2]:
dataset = MNISTSuperpixels('mnist_data/',NormalizeFeatures())

In [3]:
print('There are this many graphs (images) in the dataset: '+str(len(dataset)))

There are this many graphs (images) in the dataset: 60000


In [4]:
dataset[2].x.shape

torch.Size([75, 1])

In [5]:
dataset[2].edge_index

tensor([[ 0,  0,  0,  ..., 74, 74, 74],
        [ 0,  3,  4,  ..., 65, 66, 68]])

In [6]:
dataset = dataset.shuffle()
train_dataset = dataset[:50000]
test_dataset = dataset[50000:]

In [7]:
from torch_geometric.data import DataLoader

In [18]:
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=True)
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)

In [9]:
for step,data in enumerate(train_loader):
    print(data[0].y)

tensor([2])
tensor([8])
tensor([1])
tensor([9])
tensor([6])
tensor([7])
tensor([0])
tensor([7])
tensor([2])
tensor([9])
tensor([8])
tensor([5])
tensor([0])
tensor([2])
tensor([4])
tensor([8])
tensor([7])
tensor([8])
tensor([7])
tensor([3])
tensor([9])
tensor([6])
tensor([7])
tensor([2])
tensor([0])
tensor([6])
tensor([5])
tensor([1])
tensor([4])
tensor([6])
tensor([7])
tensor([4])
tensor([3])
tensor([6])
tensor([8])
tensor([1])
tensor([2])
tensor([6])
tensor([9])
tensor([3])
tensor([6])
tensor([7])
tensor([0])
tensor([6])
tensor([3])
tensor([1])
tensor([9])
tensor([2])
tensor([7])
tensor([1])
tensor([9])
tensor([5])
tensor([0])
tensor([8])
tensor([6])
tensor([5])
tensor([2])
tensor([4])
tensor([1])
tensor([1])
tensor([0])
tensor([8])
tensor([5])
tensor([8])
tensor([1])
tensor([9])
tensor([6])
tensor([8])
tensor([8])
tensor([2])
tensor([6])
tensor([3])
tensor([6])
tensor([0])
tensor([7])
tensor([0])
tensor([0])
tensor([4])
tensor([9])
tensor([7])
tensor([1])
tensor([9])
tensor([0])
tens

tensor([3])
tensor([8])
tensor([8])
tensor([2])
tensor([7])
tensor([7])
tensor([4])
tensor([3])
tensor([6])
tensor([1])
tensor([6])
tensor([1])
tensor([0])
tensor([9])
tensor([3])
tensor([1])
tensor([0])
tensor([5])
tensor([1])
tensor([7])
tensor([6])
tensor([2])
tensor([5])
tensor([6])
tensor([9])
tensor([7])
tensor([4])
tensor([2])
tensor([8])
tensor([0])
tensor([1])
tensor([1])
tensor([5])
tensor([4])
tensor([8])
tensor([4])
tensor([0])
tensor([7])
tensor([1])
tensor([1])
tensor([7])
tensor([1])
tensor([2])
tensor([5])
tensor([4])
tensor([1])
tensor([1])
tensor([3])
tensor([4])
tensor([6])
tensor([2])
tensor([0])
tensor([5])
tensor([1])
tensor([5])
tensor([9])
tensor([5])
tensor([2])
tensor([1])
tensor([5])
tensor([8])
tensor([8])
tensor([0])
tensor([6])
tensor([0])
tensor([6])
tensor([0])
tensor([4])
tensor([9])
tensor([4])
tensor([9])
tensor([5])
tensor([3])
tensor([0])
tensor([6])
tensor([8])
tensor([7])
tensor([6])
tensor([2])
tensor([1])
tensor([1])
tensor([0])
tensor([2])
tens

In [10]:
### Model time

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,global_mean_pool

class Model(nn.Module):
    def __init__(self,hidden_channels):
        super(Model,self).__init__()
        self.conv1 = GCNConv(1,16)
        self.conv2 = GCNConv(16,16)
        self.conv3 = GCNConv(16,16)
        self.linear = nn.Linear(16,10)
    
    def forward(self,x,edge_index,batch):
        x = self.conv1(x,edge_index)
        x = x.relu()
        x = self.conv2(x,edge_index)
        x = x.relu()
        x = self.conv3(x,edge_index)
        
        x = global_mean_pool(x,batch)
      #  print(x.shape)
        #print(x[0])
        #print(x[0].shape)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.linear(x)
        return x
    def get_graph_embedding(self,x,edge_index,batch):
        x = self.conv1(x,edge_index)
        x = x.relu()
        x = self.conv2(x,edge_index)
        x = x.relu()
        x = self.conv3(x,edge_index)
        
        x = global_mean_pool(x,batch)
        return x

In [58]:
model = Model(16)

num_epochs = 10
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [63]:
def train_epoch():
    total_loss_epoch = 0
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data.x,data.edge_index,data.batch)
        loss = criterion(output,data.y)
        
        loss.backward()
        optimizer.step()
        total_loss_epoch+=loss
       # print(loss)
    return total_loss_epoch/len(train_loader)

def test_epoch():
    correct = 0
    for data in test_loader:
        output = model(data.x,data.edge_index,data.batch)
        preds = output.argmax(dim=1)
        correct += int((preds==data.y).sum())
    return correct/len(test_loader.dataset)

In [64]:
for epoch in range(num_epochs):
    train_loss = train_epoch()
    test_acc = test_epoch()
    print('Train Loss: '+str(train_loss)+'..Test Acc: '+str(test_acc))

Train Loss: tensor(2.2984, grad_fn=<DivBackward0>)..Test Acc: 0.1134
Train Loss: tensor(2.2743, grad_fn=<DivBackward0>)..Test Acc: 0.155
Train Loss: tensor(2.2366, grad_fn=<DivBackward0>)..Test Acc: 0.1799
Train Loss: tensor(2.1675, grad_fn=<DivBackward0>)..Test Acc: 0.2286
Train Loss: tensor(2.0940, grad_fn=<DivBackward0>)..Test Acc: 0.2481
Train Loss: tensor(2.0530, grad_fn=<DivBackward0>)..Test Acc: 0.259
Train Loss: tensor(2.0282, grad_fn=<DivBackward0>)..Test Acc: 0.249
Train Loss: tensor(2.0094, grad_fn=<DivBackward0>)..Test Acc: 0.2634
Train Loss: tensor(1.9982, grad_fn=<DivBackward0>)..Test Acc: 0.2667
Train Loss: tensor(1.9900, grad_fn=<DivBackward0>)..Test Acc: 0.2696
